In [50]:
# Übersicht über den Datensatz verschaffen

import pandas as pd

df = pd.read_csv("Daten.csv")
df.head()

# Die video_id sowie channel_id enthält keine Informationen, die für die Analyse relevant sind und wird daher entfernt

df = df.drop("video_id", axis=1)
df = df.drop("channel_id", axis=1)

,published_at,length_title,length_description,liveBroadcastContent,length_tags,category,duration,dimension,definition,caption,licensedContent,privacyStatus,view_count,like_count,favorite_count,comment_count,channel_view_count,channel_sub_count,channel_video_count,channel_country
0,2024-01-13T11:00:19Z,53,1019,none,3.0,People & Blogs,PT29M44S,2d,hd,True,True,public,419706,16313,0,607,224573224,1450000,215,JP
1,2024-01-06T12:00:14Z,75,1151,none,3.0,People & Blogs,PT34M8S,2d,hd,True,True,public,698272,25174,0,1379,224573224,1450000,215,JP
2,2022-10-26T21:01:13Z,22,0,none,NaN,Film & Animation,PT15S,2d,hd,False,True,public,4190814,252938,0,1795,551924556,381000,302,US
3,2022-08-06T20:02:07Z,22,0,none,NaN,Film & Animation,PT15S,2d,hd,False,True,public,3621409,241229,0,1614,551924556,381000,302,US
4,2022-09-20T19:20:13Z,62,0,none,NaN,People & Blogs,PT47S,2d,hd,False,True,public,17097671,1498921,0,7420,2917583937,6480000,470,NaN


In [51]:

# Es wird eine weitere Spalte eingefügt, die aus "published at" die Dauer bis zum heutigen Tag berechnet, um die Aktualität der Videos zu berücksichtigen

from datetime import datetime, timezone

df['published_at'] = pd.to_datetime(df['published_at'], utc=True)

# Aktuelles Datum mit der gleichen Zeitzone wie 'published_at'
heutiger_tag = datetime.now(timezone.utc)

# Berechne die Differenz zwischen 'published_at' und dem heutigen Tag
df['days_online'] = (heutiger_tag - df['published_at']).dt.days

In [52]:
# um für mögliche Saisonale Effekte zu berücksichtigen, wird die Spalte "published_at" in einer weiteren Spalte in Monate umgewandelt
df['published_month'] = df['published_at'].dt.month

In [53]:
# Die Spalte "duration" wird in eine Spalte "duration_sec" umgewandelt, um die Dauer der Videos in Sekunden zu erhalten
df['duration_sec'] = pd.to_timedelta(df['duration']).dt.total_seconds()
# die Kommastellen werden entfernt
df['duration_sec'] = df['duration_sec'].astype(int)
# Die Spalte "duration" wird entfernt
df = df.drop("duration", axis=1)

In [54]:
# die unterschiedlichen Werte der Spalte "Favorite Count", sowie "dimension" und "definition" werden angezeigt unt entfernt, da sie keine neue Information liefern
df['favorite_count'].unique()
df['dimension'].unique()
df['definition'].unique()

df = df.drop("favorite_count", axis=1)
df = df.drop("dimension", axis=1)
df = df.drop("definition", axis=1)

In [55]:
# da die Spalte "published_at" bereits in den Spalten published_month und days_online enthalten ist, wird sie entfernt
df = df.drop("published_at", axis=1)

In [56]:
# die Spalte "length_tags" ist sehr unvollständig und wird daher entfernt
df = df.drop("length_tags", axis=1)

In [57]:
# die Spalte "live_broadcasted" enthält keine unterschiedlichen Informationen und wird daher entfernt
df = df.drop("liveBroadcastContent", axis=1)

In [58]:
# Eine beobachtung mit "channel_country" NaN wird entfernt
df = df.dropna(subset=['channel_country'])

,length_title,length_description,category,caption,licensedContent,privacyStatus,view_count,like_count,comment_count,channel_view_count,channel_sub_count,channel_video_count,channel_country,days_online,published_month,duration_sec
0,53,1019,People & Blogs,True,True,public,419706,16313,607,224573224,1450000,215,JP,9,1,1784
1,75,1151,People & Blogs,True,True,public,698272,25174,1379,224573224,1450000,215,JP,16,1,2048
2,22,0,Film & Animation,False,True,public,4190814,252938,1795,551924556,381000,302,US,452,10,15
3,22,0,Film & Animation,False,True,public,3621409,241229,1614,551924556,381000,302,US,533,8,15
5,82,1074,People & Blogs,True,True,public,2857435,60915,2094,224573224,1450000,215,JP,44,12,1474


In [81]:
# Nun muss der Datensatz noch für die Analyse mit linerer Regression vorbereitet werden: Dazu müssen nicht numerische Werte in Dummy Variablen abgebildet werden.

# Die Spalte "category" wird in Dummy Variable-Spalten aufgeteilt, die dann 1 sind, wenn das Video in der jeweiligen Kategorie ist und 0, wenn nicht, die Spalten werden so benannt wie die Kategorien
df_analysis = pd.get_dummies(df, columns=['category'], prefix="cat")

In [82]:
# Die Spalte "channel_country" wird in Dummy Variable-Spalten aufgeteilt, die dann 1 sind, wenn das Video aus dem jeweiligen Land ist und 0, wenn nicht, die Spalten werden so benannt wie die Länder
df_analysis = pd.get_dummies(df_analysis, columns=['channel_country'], prefix="country")

In [83]:
# Spalte "caption" wird in Dummy Variable-Spalten aufgeteilt, die dann 1 sind, wenn das Video Untertitel hat und 0, wenn nicht
df_analysis = pd.get_dummies(df_analysis, columns=['caption'], prefix="caption")

In [84]:
# Spalte "licensedContent" wird in Dummy Variable-Spalten aufgeteilt, die dann 1 sind, wenn das Video lizensierten Inhalt enthält und 0, wenn nicht
df_analysis = pd.get_dummies(df_analysis, columns=['licensedContent'], prefix="licensed")

In [ ]:
# Spalte "privacyStatus" wird in Dummy Variable-Spalten aufgeteilt, die dann 1 sind, wenn das Video öffentlich ist und 0, wenn nicht
df_analysis = pd.get_dummies(df_analysis, columns=['privacyStatus'], prefix="privacy")

In [87]:
# Spalten "caption_False" und "licensed_False" werden entfernt, da sie keine neuen Informationen liefern
df_analysis = df_analysis.drop("caption_False", axis=1)
df_analysis = df_analysis.drop("licensed_False", axis=1)

In [88]:
df_analysis.head()

,length_title,length_description,view_count,like_count,comment_count,channel_view_count,channel_sub_count,channel_video_count,days_online,published_month,...,country_TR,country_TW,country_UA,country_UG,country_US,country_VN,country_ZA,caption_True,licensed_True,privacy_public
0,53,1019,419706,16313,607,224573224,1450000,215,9,1,...,0,0,0,0,0,0,0,1,1,1
1,75,1151,698272,25174,1379,224573224,1450000,215,16,1,...,0,0,0,0,0,0,0,1,1,1
2,22,0,4190814,252938,1795,551924556,381000,302,452,10,...,0,0,0,0,1,0,0,0,1,1
3,22,0,3621409,241229,1614,551924556,381000,302,533,8,...,0,0,0,0,1,0,0,0,1,1
5,82,1074,2857435,60915,2094,224573224,1450000,215,44,12,...,0,0,0,0,0,0,0,1,1,1
